<a href="https://colab.research.google.com/github/playeredlc/treinamento-h2ia/blob/master/Meta-heuristicas/algoritmo_genetico_mochila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Metaheuristics
# Solving the Knapsack problem using Genetic Algorithm

In [2]:
import math
import numpy as np

In [3]:
class Knapsack:
  def __init__(self, weights, values, weight_limit):
    self.weights = weights
    self.values = values
    self.weight_limit = weight_limit
    self.items_amount = len(self.weights)
    self.solution = None

    def set_solution(self, solution):
      self.solution = solution

In [4]:
class Chromosome:
  def __init__(self, size, genes=None):
    self.size = size
    
    if(genes == None):
      self.genes = self.generate_random_genes()
    else:
      self.set_genes(genes)
    
    self.fitness = None

  def generate_random_genes(self):
    genes = np.zeros(self.size, dtype=int)
    for i in range(self.size):
      genes[i] = np.random.randint(0, 2)
    
    return genes
  
  def set_genes(self, genes):
    self.genes = np.array(genes)

  def set_fitness(self, fitness):
    self.fitness = fitness

  def mutate(self, num_genes=1):
    for _ in range(num_genes):        
      random_gene = np.random.randint(0, self.size)
      if(self.genes[random_gene] == 0):
        self.genes[random_gene] = 1
      else:
        self.genes[random_gene] = 0
      

In [5]:
class Population:
  def __init__(self, population_size, individual_size, elite_size=0):
    self.population_size = population_size
    self.individual_size = individual_size
    self.individuals = self.generate_population()
    self.elite_size = elite_size
    self.num_generations = 0

  def generate_population(self):
    population = []
    for _ in range(self.population_size):
      new_individual = Chromosome(self.individual_size)
      population.append(new_individual)
    
    return population

  def rank_individuals(self):
    self.individuals.sort(key=lambda x: x.fitness, reverse=True)

  def select(self, selection_rate=0.5):
    # round to even
    new_size = math.ceil(selection_rate * len(self.individuals) / 2) * 2

    self.rank_individuals()
    self.individuals = self.individuals[:new_size]

  def crossover(self, chr1, chr2, mutation_rate=0.01):
    cross_point = np.random.randint(1, self.individual_size)
    genes1 = chr1[:cross_point] + chr2[cross_point:]
    child1 = Chromosome(self.individual_size, genes1)

    genes2 = chr2[:cross_point] + chr1[cross_point:]
    child2 = Chromosome(self.individual_size, genes2)

    # mutate if chromosome was chosen
    mutation = np.random.choice([True, False], p=[mutation_rate, 1-mutation_rate])
    if(mutation):
      child1.mutate()
      child2.mutate()

    return child1, child2

  def new_generation(self, mutation_rate=0.01):
    new_gen = []    
    np.random.shuffle(self.individuals)

    # crossover in pairs
    for i in range(0, len(self.individuals)-1, 2):
      chr1 = list(self.individuals[i].genes)
      chr2 = list(self.individuals[i+1].genes)

      child1, child2 = self.crossover(chr1, chr2, mutation_rate)

      new_gen.append(child1)
      new_gen.append(child2)
    
    self.individuals += new_gen
    self.population_control()
    self.num_generations += 1

  def population_control(self):
    if(len(self.individuals) > self.population_size):
      self.individuals = self.individuals[:self.population_size]


In [6]:
def objective_function(knapsack, state):
  total_value = 0
  total_weight = 0
  
  for i in range(knapsack.items_amount):
    if(state[i] != 0):
      total_value += knapsack.values[i]
      total_weight += knapsack.weights[i]

  return total_value, total_weight

In [7]:
# knapsack definition
weights = np.array([63, 21, 2, 32, 13, 80, 19, 37, 56, 41, 14, 8, 32, 42, 7])
values = np.array([13, 2, 20, 10, 7, 14, 7, 2, 2, 4, 16, 17, 17, 3, 21])
weight_limit = 275

knapsack = Knapsack(weights, values, weight_limit)


In [8]:
def genetic_algorithm(knapsack):
  max_gen = 5
  population_size = 800
  individual_size = knapsack.items_amount
  mutation_rate = 0.05

  population = Population(population_size, individual_size)  
  # calculate fitness
  for individual in population.individuals:
    fit, _ = objective_function(knapsack, individual.genes)
    individual.set_fitness(fit)

  for i in range(max_gen):    
    print(f'Starting Generation {i+1}...')

    # select the best half of the population and discard the other half
    population.select()

    # repopulate with new individuals using crossover
    population.new_generation(mutation_rate)

    # recalculate fitness
    for individual in population.individuals:
      fit, _ = objective_function(knapsack, individual.genes)
      individual.set_fitness(fit)
  
  # find and return best valid (not overweighted) individual
  population.rank_individuals()
  for individual in population.individuals:
    v, w = objective_function(knapsack, individual.genes)
    if(w <= knapsack.weight_limit):
      return population, individual

  return None, None


In [9]:
population, solution = genetic_algorithm(knapsack)
v, w = objective_function(knapsack, solution.genes)

print(f'\nNumber of generations: {population.num_generations}')
if(solution):
  print(f'Best individual: {solution.genes}')
  print(f'Solution value: {v}')
  print(f'Solution weight: {w}')
else:
  print('No valid solution was found.')

# TODO
# consider weight while ranking individuals!?
#

Starting Generation 1...
Starting Generation 2...
Starting Generation 3...
Starting Generation 4...
Starting Generation 5...

Number of generations: 5
Best individual: [1 0 1 1 1 1 1 0 0 0 1 1 1 0 1]
Solution value: 142
Solution weight: 270
